# Manual curation of fermentative pathways and glycolysis

In [1]:
from reframed import load_cbmodel, save_cbmodel, Protein, GPRAssociation, simplify, Environment
from reframed.io.sbml import parse_gpr_rule
from copy import deepcopy

In [2]:
model = load_cbmodel('../models/RcH10_v1.xml')

### Fermentative pathways 

In [3]:
model.remove_reaction('R_ME1')
model.add_reaction_from_str('R_FNRR: M_fdxrd_c + M_h_c + M_nad_c <-> M_fdxo_2_2_c + M_nadh_c')
model.add_reaction_from_str('R_FNRR2: M_fdxrd_c + M_h_c + M_nadp_c <-> M_fdxo_2_2_c + M_nadph_c')
model.add_reaction_from_str('R_H2td: M_h2_c <-> M_h2_e')

prot = Protein()
prot.genes=['G_WP_015925963_1']
gpr = GPRAssociation()
gpr.proteins=[prot]
model.set_gpr_association('R_FNRR2', gpr)

### Cofactors in glycolysis

In [4]:
model.add_reaction_from_str('R_PFK_ppi: M_g6p_c + M_ppi_c <-> M_fdp_c + M_h_c + M_pi_c')
model.reactions['R_PFK_ppi'].name = 'diphosphate--fructose-6-phosphate 1-phosphotransferase'

prot = Protein()
prot.genes=['G_WP_015925658_1']
gpr = GPRAssociation()
gpr.proteins=[prot]
model.set_gpr_association('R_PFK_ppi', gpr)

In [5]:
model.add_reaction_from_str('R_HEX1_gtp: M_glc__D_c + M_gtp_c <-> M_g6p_c + M_gdp_c + M_h_c')
model.reactions['R_HEX1_gtp'].name = 'GTP dependent hexokinase'

prot = Protein()
prot.genes=['G_WP_015926569_1']
gpr = GPRAssociation()
gpr.proteins=[prot]
model.set_gpr_association('R_HEX1_gtp', gpr)

In [6]:
model.add_reaction_from_str('R_GALK2_gtp: M_a_gal__D_c + M_gtp_c --> M_gal1p_c + M_gdp_c + M_h_c')
model.reactions['R_GALK2_gtp'].name = 'alpha Galactokinase GTP dependent'

prot = Protein()
prot.genes=['G_WP_015926586_1']
gpr = GPRAssociation()
gpr.proteins=[prot]
model.set_gpr_association('R_GALK2_gtp', gpr)

In [7]:
model.add_reaction_from_str('R_GALKr_gtp: M_gal_c + M_gtp_c <-> M_gal1p_c + M_gdp_c + M_h_c')
model.reactions['R_GALKr_gtp'].name = 'Galactokinase GTP dependent'

prot = Protein()
prot.genes=['G_WP_015926586_1']
gpr = GPRAssociation()
gpr.proteins=[prot]
model.set_gpr_association('R_GALKr_gtp', gpr)

In [8]:
model.reactions.R_PFK.gpr.proteins.pop(1)
model.reactions.R_HEX1.gpr.proteins.pop(0)
model.remove_reactions(['R_CD6P', 'R_r0191', 'R_ID6P', 'R_PFK', 'R_GALK2', 'R_GALKr'])

### Mis-annotated transporters leading to false negative essentiality predictions

In [9]:
model.remove_reactions(['R_HEX1', 'R_XYLI2','R_GLUKA_1','R_MNLDHr'])
model.reactions.R_CEPA.reversible=False
model.reactions.R_CEPA.lb=0

### Other mis-annotated suger transporters

In [10]:
model.remove_reactions(['R_ARBabcpp', 'R_GLCabcpp', 'R_XYLabcpp'])

/Volumes/Daniel/git/reframed/reframed/core/model.py:471: UserWarning: No such reaction R_ARBabcpp
  warn(f"No such reaction {r_id}")


In [11]:
rxns = [rxn for rxn in model.get_reactions_by_gene('G_WP_041706401_1') if len(model.reactions[rxn].get_genes())==1]
model.remove_reactions(rxns)

In [12]:
model.add_reaction_from_str('R_GALabc: M_atp_c + M_gal_e + M_h2o_c <-> M_adp_c + M_gal_c + M_h_c + M_pi_c')
gpr = deepcopy(model.reactions.R_GALabcpp.gpr)
model.set_gpr_association('R_GALabc', gpr)

In [13]:
pts_rxns = [rxn for rxn in [rxn for rxn in model.reactions if rxn.endswith('pts')]]
model.remove_reactions(pts_rxns)

In [14]:
model.add_reaction_from_str('R_GALUi: M_g1p_c + M_h_c + M_utp_c <-> M_ppi_c + M_udpg_c')
prot = Protein()
prot.genes=['G_WP_015926747_1']
gpr = GPRAssociation()
gpr.proteins=[prot]
model.set_gpr_association('R_GALUi', gpr)

### Remove ABC transporters using both cytosol and periplasm

In [15]:
duplicated = [r_id for r_id in model.reactions if r_id.endswith('abcpp') and r_id[:-2] in model.reactions]

for r_id in duplicated:
    gpr1 = model.reactions[r_id].gpr
    gpr2 = model.reactions[r_id[:-2]].gpr
    gpr_str = f'({gpr1}) or ({gpr2})'

    gpr = parse_gpr_rule(gpr_str)
    model.set_gpr_association(r_id[:-2], gpr)
    
model.remove_reactions(duplicated)

## Remove blocked reactions and orphaned genes

In [16]:
Environment.complete(model, inplace=True)
simplify(model, inplace=True)

## Save model

In [17]:
save_cbmodel(model, '../models/RcH10_v2.xml')